In [389]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
!pip install selenium
!pip install webdriver-manager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

In [503]:
# Grab the Intial Page showing the First Group of Comapnies
page = requests.get("https://canada.constructconnect.com/dcn/construction-directory/search?ccpage=1&perpage=16&phrase&category&location#results").text

In [504]:
# Parse the Page and create BS Object
soup = BeautifulSoup(page, "html.parser")

In [505]:
# Find the Section that has the listed Companies on the Page
allSections = soup.find_all('section')
for index, section in enumerate(allSections):
    if("Search Results" in str(section)):
        find_section = index
print(find_section)

4


In [506]:
# Find the Links to each Company
links = allSections[find_section].find_all('a')
links[0:5]

[<a class="promos-cover promos-cover--visible" href="/dcn/construction-directory/22005C29-8865-4FAD-B82C-46A622ED5EFD" title="ConstructConnect Details"></a>,
 <a class="link-tertiary" href="/dcn/construction-directory/22005C29-8865-4FAD-B82C-46A622ED5EFD" title="ConstructConnect Details">See company details<i class="icon icon-angle--right"></i></a>,
 <a class="promos-cover promos-cover--visible" href="/dcn/construction-directory/6B2F4E69-B21F-4B85-AF84-FC02CB71CBDC" title="#1 Prime Contracting Details"></a>,
 <a class="link-tertiary" href="/dcn/construction-directory/6B2F4E69-B21F-4B85-AF84-FC02CB71CBDC" title="#1 Prime Contracting Details">See company details<i class="icon icon-angle--right"></i></a>,
 <a class="promos-cover promos-cover--visible" href="/dcn/construction-directory/AEADEAF6-1E99-462C-8605-B0B462E7DD60" title="1187839 Ontario Ltd Details"></a>]

In [523]:
# There are extra links and Duplicates, so exclude the one's with HTTP and get rid of deuplicates
hrefs = []
for link in allSections[find_section].find_all('a'):
    if ('http' not in link):
        hrefs.append(link.get('href'))

# Get Rid of the Duplicates
hrefs = [*set(hrefs)]
# Get Rid of 'http' hrefs
hrefs = [ x for x in z if "http" not in x ]
hrefs[0]

'/dcn/construction-directory/004f3b9a-d591-46d4-8175-57d653827d61'

In [529]:
# A Function to Clean the data for the single company
def clean_data(initial_data):
    
    # get rid of the \n and strip blank spaces
    initial_data = {x.replace('\n', '') for x in initial_data}
    initial_data = [elem.strip() for elem in new_set]
    
    # Split the Address, strip the spaces    
    address = initial_data[1]
    split_list = address.split(',')
    [i.strip() for i in split_list]

    # Split the City and Zip Code and then delete the spaces
    a = split_list[2]
    a = re.split(r'\s{4,}', a)
    del a[0]

    # Delete the old Addresses
    del split_list[2]
    del initial_data[1]

    # Add the data back together and strip address space 
    initial_data = initial_data + split_list + a
    initial_data = [i.strip() for i in initial_data]
    
    return initial_data

In [535]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

# Will Create Loop of ALL Pages to then Loop Companies on each Page
# Will Change to Loop of Companies on Single Page
data = []
final_data = []
for i,url in enumerate(hrefs):
    driver.get('https://canada.constructconnect.com/dcn/construction-directory/search?ccpage=1&perpage=16&phrase&category&location#results')

    firstLink = driver.find_element("xpath", f"//a[contains(@href, '{hrefs[i]}')]").click()

    # Get the Contact/Address info for Company (I used BeautifulSoup becasue it is more familiar to me)
    page = requests.get("https://canada.constructconnect.com" + hrefs[i])
    soup2 = BeautifulSoup(page.text)
    for ulTag in soup2.find_all('ul', {'class': 'company-details'}):
        for liTag in ulTag.find_all('li'):
            data.append(liTag.text)



    # Call Function to clean the data
    data = clean_data(data)
    new_data.append(data)
    # Get Company Name
#     company_name = soup2.select('h1.company-name')[0].text.strip()
    
    
    final_data.append(new_data)

In [536]:
final_data

[['Thermal and Moisture Protection',
  'Sub-Contractors',
  '416-679-8911',
  '32 Goodmark Place Unit 17',
  'Etobicoke',
  'Ontario',
  'M9W 6J4',
  ['Thermal and Moisture Protection',
   'Sub-Contractors',
   '416-679-8911',
   '32 Goodmark Place Unit 17',
   'Etobicoke',
   'Ontario',
   'M9W 6J4',
   '\n\n  General Contractors\n  \n',
   '\n\n  General Building\n  \n',
   '\n    27 Nashville Ave, Suite 1, \n    York, \n    Ontario \n    M6M 1J2\n  ',
   '\n416-825-9037\nFax (888) 521-3334\n'],
  ['Thermal and Moisture Protection',
   'Sub-Contractors',
   '416-679-8911',
   '32 Goodmark Place Unit 17',
   'Etobicoke',
   'Ontario',
   'M9W 6J4',
   '\n\n  General Contractors\n  \n',
   '\n\n  Civil\n  \n',
   '\n    1160 Waterdown Rd Unit 1, \n    Burlington, \n    Ontario \n    L7T 4A8\n  ',
   '\n905-633-6938\nFax 905-633-7132\n'],
  ['Thermal and Moisture Protection',
   'Sub-Contractors',
   '416-679-8911',
   '32 Goodmark Place Unit 17',
   'Etobicoke',
   'Ontario',
   'M9W 6

In [297]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

url = 'https://canada.constructconnect.com/dcn/construction-directory/004f3b9a-d591-46d4-8175-57d653827d61'

driver.get(url)
driver.page_source




# driver.find_element('xpath', "//a[@title = 'company-details']")

'<html class="no-js" lang="en-US"><!--<![endif]--><head>\n  <meta charset="UTF-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge"><script gtm="GTM-TNN9JTG" type="text/javascript" async="" src="https://www.google-analytics.com/gtm/optimize.js?id=GTM-T5BJBQS"></script><script src="https://js-agent.newrelic.com/nr-spa-1216.min.js"></script><script async="" src="https://snap.licdn.com/li.lms-analytics/insight.old.min.js"></script><script type="text/javascript" async="" src="https://snap.licdn.com/li.lms-analytics/insight.min.js"></script><script type="text/javascript" async="" src="https://www.google-analytics.com/analytics.js"></script><script src="https://js.hsadspixel.net/fb.js" type="text/javascript" id="hs-ads-pixel-2347101" data-ads-portal-id="2347101" data-ads-env="prod" data-loader="hs-scriptloader" data-hsjs-portal="2347101" data-hsjs-env="prod" data-hsjs-hublet="na1"></script><script src="https://js.hsleadflows.net/leadflows.js" type="text/javascript" id="LeadFlows-23471